In [1]:
"""Crops_Predictor.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/18cEe7mlmxLkv0Q2eGXTlqlaZSnDPAQKC
"""

'Crops_Predictor.ipynb\n\nAutomatically generated by Colaboratory.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/18cEe7mlmxLkv0Q2eGXTlqlaZSnDPAQKC\n'

importing data

In [ ]:
import pandas as pd
import certifi
import urllib3
# import pickle
from io import BytesIO

In [ ]:
# Set the SSL certificates for requests
http = urllib3.PoolManager(
    cert_reqs='CERT_REQUIRED',
    ca_certs=certifi.where()
)

In [ ]:
# Read the Excel file using requests and BytesIO
url = "https://github.com/shreyas023/HackX/raw/main/Crop_prediction.xlsx"
response = http.request('GET', url)
content = BytesIO(response.data)
crop = pd.read_excel(content, engine='openpyxl')

In [ ]:
# Display the DataFrame
print(crop)

exploring data

In [ ]:
crop.head()

In [ ]:
crop.describe()

In [ ]:
crop.columns

In [ ]:
crop.shape

In [ ]:
crop.isnull().sum()

In [ ]:
crop.nunique()

In [ ]:
crop.corr(numeric_only=True)

In [ ]:
crop.info()

preprocessing the data

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
le = LabelEncoder()

In [ ]:
crop['Region'] = le.fit_transform(crop['Region'])
crop['WeatherCondition'] = le.fit_transform(crop['WeatherCondition'])
crop['Bulk Nutrient'] = crop['Bulk Nutrient'].astype(str)
crop['SeedQuality'] = crop['SeedQuality'].astype(str)
crop['Crop Cultivation'] = le.fit_transform(crop['Crop_Cultivation'])

In [ ]:
crop.head(10)

visualising data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.pairplot(crop)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Identify non-numeric columns
non_numeric_columns = crop.select_dtypes(exclude='number').columns

In [ ]:
# Drop non-numeric columns and convert the remaining data to numeric
crop_numeric = crop.drop(non_numeric_columns, axis=1).apply(pd.to_numeric, errors='coerce')

In [ ]:
# Plot heatmap
plt.figure(figsize=(10, 10))
sns.heatmap(crop_numeric.corr(), annot=True, cmap="YlGnBu")
plt.show()

train test split

In [ ]:
# Convert 'Bulk Nutrient' and 'Seed Quality' to strings
crop['Bulk Nutrient'] = crop['Bulk Nutrient'].astype(str)
crop['SeedQuality'] = crop['SeedQuality'].astype(str)

In [ ]:
# Train-Test Split
X = crop.drop(['Crop Cultivation', 'Crop_Cultivation', 'Irrigation Method'], axis=1)
y1 = crop['Crop Cultivation']
y2 = crop['Irrigation Method']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y1, random_state=1, test_size=0.2)
X1_train, X1_test, y1_train, y1_test = train_test_split(X, y2, random_state=1, test_size=0.2)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y1,random_state=1,test_size=0.2)

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

training

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Apply label encoding to 'Crop Cultivation' (target variable)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
# Define a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', X_train.select_dtypes(exclude='object').columns),
        ('cat', OneHotEncoder(), X_train.select_dtypes(include='object').columns)
    ])

In [ ]:
# Create a pipeline with the preprocessor and the model
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', model)])

In [ ]:
# Fit the pipeline on the training data
pipeline.fit(X_train, y_train_encoded)

testing

In [ ]:
# Assuming pipeline is your pre-processing pipeline
y_pred = pipeline.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
acc1 = accuracy_score(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

predicting

In [ ]:
crop.iloc[[25]]

In [ ]:
res1 = pipeline.predict(X.iloc[[25]])

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X,y2,random_state=1,test_size=0.2)

In [ ]:
X1_train.shape,X1_test.shape,y1_train.shape,y1_test.shape

In [ ]:
pipeline.fit(X1_train,y1_train)

In [ ]:
y1_pred = pipeline.predict(X1_test)

In [ ]:
acc2 = accuracy_score(y1_test,y1_pred)

In [ ]:
print(classification_report(y1_test,y1_pred))

In [ ]:
crop.iloc[[25]]

In [ ]:
res2 = pipeline.predict(X.iloc[[25]])

conclusion

In [ ]:
print("Suggested Cultivation :",res1)
print("Suggested Irrigation :",res2)

In [ ]:
average = (acc1+acc2)/2
print("Accuracy of model = ",average*100,"%")

In [ ]:
import joblib

In [ ]:
# Assuming 'pipeline' is your scikit-learn pipeline
joblib.dump(pipeline, 'model_pipeline.pkl')

In [ ]:
loaded_pipeline = joblib.load('model_pipeline.pkl')

In [ ]:
!pip3 install pickle

pickle.dump(log_reg,open('model.pkl','wb'))
model=pickle.load(open('model.pkl','rb'))